# Adverserial Examples

Consider an input $x$ with true label $y$, to a machine learning classifier model.  An **adversarial example** for this model would be an input $\tilde{x}$ whose values are only slightly different from $x$ such that the model will misclassify it as a different label $\tilde{y}$.

The initial thought was that these examples had something to do with the nonlinearity inherent in complex machine learning models.   However, the paper ["EXPLAINING AND HARNESSING
ADVERSARIAL EXAMPLES"](https://arxiv.org/pdf/1412.6572.pdf) by Ian J. Goodfellow, Jonathon Shlens and Christian Szegedy of Google, indicate that in fact this behaviour comes about due to the following:
*  the **linear** behaviour of such networks
*  the **high dimensionality** of the input spaces they operate in

They also show that this leads to a relatively easy method for generating such examples.   In this workbook we will follow their prescription.


## Get the data
Here we will use the standard MNIST sample included with Keras.

In [ ]:
import keras
import tensorflow
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

short = False
if short:
    train_images = train_images[:7000,:]
    train_labels = train_labels[:7000]
    test_images = test_images[:3000,:]
    test_labels = test_labels[:3000]
#
print("Train info",train_images.shape, train_labels.shape)
print("Test info",test_images.shape, test_labels.shape)
train_images = train_images.reshape((train_images.shape[0],784))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((test_images.shape[0],784))
test_images = test_images.astype('float32')/255
from keras.utils import to_categorical

train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)


## A fully connected network (FCN) MNIST Classifier
The paper states that they can easily generate adverserial examples using a **shallow** MNIST classifier.   In this context, "shallow" typically means 1 hidden layer, and usually just refers to fully connected networks.   "Deep learning" uses multiple hidden layers and pooling techniques, and often refers to models which **learn** their features: convolutional neural networks are thus "deep".

Given that, let's build a very simple single layer FCN.  Below we will also apply this to a "deep" model.

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(400,activation='relu',input_shape=(28*28,)))
network.add(layers.Dense(10,activation='softmax'))

print("summary",network.summary())

network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#
history = network.fit(train_images,train_labels_cat,epochs=5,batch_size=128,validation_data=(test_images,test_labels_cat))


## Helper display function
We are going to want to display both are starting **true** example, as well as the **adverserial** example we will generate from that true example, so let's define a helper display function.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot_img(img):
    img = img.reshape(28,28)
    plt.imshow(img)
    plt.show()


## Quick test of the network
Lets pass in a single example from of test examples.   

In [ ]:
import numpy as np
#
# Define the sample using the index into the numpy arrays
sample = 4
#
# Get the individual prediction for a single sample in the test set
test_image = test_images[sample].reshape(1,784)
test_label = test_labels[sample]
predictions = network.predict(test_image)
#
# Get the max probability
prob = np.max(predictions, axis = 1)
#
# Get the predicted classes for each row
predLabel = np.argmax(predictions, axis = 1)
print("True label:                 ",test_label)
print("Predicted label:            ",predLabel[0])
print("Predicted probability:      ",prob[0])
print("Predictions for all classes:",predictions)
#
# Now display
plot_img(test_image)


## Generating Adverserial images
To generate adverserial images, we will start with a test image, and perturb it by a small amount $\eta$.   How do we determine $\eta$?   For our MNIST images, they are encoded using 8-bit precision, with a maximum of 255.   There we expect that changes to an image of the scale less than $\eta$=1/255=0.004 should not be noticeable.  Formally, we will set:
$$\tilde{x} = x + \eta$$
Looking at how this might interact with the weights $w$ in a network, we can see that:
$$w^T\tilde{x} = w^T x + w^T\eta$$
If the average value of the weights is $m$, and the dimension of the weights is $n$, then the impact of such a perturbation is $mn\eta$.  This implies that we can make many small changes to the input $x$, which leave it relatively unchanged, while imparting a large change to the output of the network.   The authors of the above paper call this "accidental steganography": steganography is the practice of concealing a file, message, image, or video within another file, message, image, or video.

The authors suggest this method to determine $\eta$.  Let θ be the parameters of the model, x the input to the model, y the targets associated with x, and J(θ, x, y) be the cost used to train the neural network, then:
$$\eta = \epsilon ~sign (\triangledown_x J(θ, x, y))$$
where the "sign" function simply returns the sign of the argument and $\epsilon$ is a free parameter which controls the size of the step.   The authors suggest numbers like 0.1-0.25, but we will use smaller values like 0.02.

To calculate the gradient of the loss with respect to the input, we will follow exactly the same idea as used in the "Deep Learning" book: section 5.4.2 "Visualizing covnet filters".

Read through the code and comments below to make sure you follow the procedure, at least conceptually.


In [ ]:
import keras.backend as K
# Initialize adversarial example with input image
sample = 5
test_image = test_images[sample].reshape((1,784))
x = test_image
x_adv = x
predictions = network.predict(x_adv)
#
# Get the max probabilites for each rows
probs = np.max(predictions, axis = 1)
#
# Get the predicted classes for each row
predclass = np.argmax(predictions, axis = 1)
print("Predictions",predictions)
print("Test label",test_label)
print("Pred class",predclass)
#
# Set variables
epochs = 15
epsilon = 0.02
#
# The target is a one-hot array of 1x10
target = np.zeros((1,10))
#
# Set the target to its true value for the real input
target[0,predclass[0]] = 1
#
# Define the loss function
loss = K.categorical_crossentropy(target, network.output)
#
# Get the gradient of the cost function with respect to the *input*
grads = K.gradients(loss, network.input)[0]
#
# This next line **defines** the "iterate" function using the "K.function" method.
# This takes the given network input and calculates the loss and the gradient.
iterate = K.function([network.input],[loss,grads])
#
# Now actually do the calculations
for i in range(epochs): 
#
# Get the loss and gradient value given our input image
    loss_value, grads_value = iterate([x_adv])
#
# Update the input - make or small changes here!
    x_adv = x_adv + epsilon*K.sign(grads_value)
#
# Now we can get the prediction given our modified image
    preds = network.predict(x_adv,steps=1)
    print(i, np.argmax(preds, axis = 1),preds)
#
# Once our predicted class is no longer the same as our starting class, jump out of the loop
    if predclass != np.argmax(preds, axis = 1)[0]:
        break
#
# Convert from a tensorflow tensor to a numpy array
x_adv = K.eval(x_adv)
#print(x_adv)
#
# Plot the starting image, the modified image, and their difference
plot_img(x)   
plot_img(x_adv)   
plot_img(x_adv-x)

## Simple Tasks
Here are some things to try:
1.  Try different example inputs - see if the adverserial examples are more difficult to generate for certain digits.
2.  Try playing with different $\epsilon$ values.

## Less Simple Task
Modify the code blocks below to make a simple CNN classifier for MNIST.   Train it and then use the same procedure as above to generate adverserial examples (that code is repeated as well below).

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()

#
# Your code here!

network.add(layers.Flatten())
network.add(layers.Dense(10,activation='softmax'))
network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
print(network.summary())
#
history = network.fit(train_images,train_labels_cat,epochs=5,batch_size=128,validation_data=(test_images,test_labels_cat))


## Generate adverserial examples for CNN
After fitting your CNN above, you will have to make a trivial change to the code below to get it to work with that network.

In [ ]:
import keras.backend as K
# Initialize adversarial example with input image
sample = 5
test_image = test_images[sample].reshape((1,784))
x = test_image
x_adv = x
predictions = network.predict(x_adv)
#
# Get the max probabilites for each rows
probs = np.max(predictions, axis = 1)
#
# Get the predicted classes for each row
predclass = np.argmax(predictions, axis = 1)
print("Predictions",predictions)
print("Test label",test_label)
print("Pred class",predclass)
#
# Set variables
epochs = 15
epsilon = 0.02
#
# The target is a one-hot array of 1x10
target = np.zeros((1,10))
#
# Set the target to its true value for the real input
target[0,predclass[0]] = 1
#
# Define the loss function
loss = K.categorical_crossentropy(target, network.output)
#
# Get the gradient of the cost function with respect to the *input*
grads = K.gradients(loss, network.input)[0]
#
# This next line **defines** the "iterate" function using the "K.function" method.
# This takes the given network input and calculates the loss and the gradient.
iterate = K.function([network.input],[loss,grads])
#
# Now actually do the calculations
for i in range(epochs): 
#
# Get the loss and gradient value given our input image
    loss_value, grads_value = iterate([x_adv])
#
# Update the input - make or small changes here!
    x_adv = x_adv + epsilon*K.sign(grads_value)
#
# Now we can get the prediction given our modified image
    preds = network.predict(x_adv,steps=1)
    print(i, np.argmax(preds, axis = 1),preds)
#
# Once our predicted class is no longer the same as our starting class, jump out of the loop
    if predclass != np.argmax(preds, axis = 1)[0]:
        break
#
# Convert from a tensorflow tensor to a numpy array
x_adv = K.eval(x_adv)
#print(x_adv)
#
# Plot the starting image, the modified image, and their difference
plot_img(x)   
plot_img(x_adv)   
plot_img(x_adv-x)